# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import joblib

import pandas as pd

import numpy as np
from azureml.train.automl import AutoMLConfig
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from train import clean_text, clean_df, preprocess_text

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:

data1 = pd.read_csv('Data/data_part1.csv')
data2 = pd.read_csv('Data/data_part2.csv')
data3 = pd.read_csv('Data/data_part3.csv')
dataset = data1.append(data2).append(data3)

dataset = clean_df(dataset)
dataset['text'] = dataset['text'].apply(clean_text)

X = preprocess_text(dataset['text'].tolist())
y = dataset['y'].to_numpy()
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.15, random_state=42)


train_dataset = pd.DataFrame(X_train)
train_dataset.columns= "X" + train_dataset.columns.astype(str)
train_dataset['y'] = y_train

val_dataset = pd.DataFrame(X_val)
val_dataset.columns= "X" + val_dataset.columns.astype(str)
val_dataset['y'] = y_val

test_dataset = pd.DataFrame(X_test)
test_dataset.columns= "X" + test_dataset.columns.astype(str)
test_dataset['y'] = y_test

if not os.path.isdir('clean_data'):
    os.mkdir('clean_data')

train_dataset.to_csv("clean_data/train_dataset.csv", index=False)
val_dataset.to_csv("clean_data/val_dataset.csv", index=False)
test_dataset.to_csv("clean_data/test_dataset.csv", index=False)



In [9]:
train_dataset.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9',
       ...
       'X278', 'X279', 'X280', 'X281', 'X282', 'X283', 'X284', 'X285', 'X286',
       'y'],
      dtype='object', length=288)

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Capstone-AutoML'

experiment=Experiment(ws, experiment_name)



In [5]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./clean_data', target_path='automldata', overwrite=True, show_progress=True)


Uploading an estimated of 3 files
Uploading ./clean_data/test_dataset.csv
Uploaded ./clean_data/test_dataset.csv, 1 files out of an estimated total of 3
Uploading ./clean_data/train_dataset.csv
Uploaded ./clean_data/train_dataset.csv, 2 files out of an estimated total of 3
Uploading ./clean_data/val_dataset.csv
Uploaded ./clean_data/val_dataset.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_ade94e21883a4312b44064e6bf1b5bf3

In [6]:
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/train_dataset.csv'))
val_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/val_dataset.csv'))

test_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/test_dataset.csv'))

    

In [7]:
run = experiment.start_logging()

In [8]:
udacity_compute_name = 'comp-capstone'

compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)


try:
    compute_target = ComputeTarget(workspace=ws, name=udacity_compute_name)
    print("Found an existing compute target.")
except:
    compute_target = ComputeTarget.create(workspace=ws, 
    name=udacity_compute_name, provisioning_configuration=compute_config)
    print("Creating a new compute target.")

compute_target.wait_for_completion(show_output=True)

Found an existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    'experiment_timeout_minutes': 30,
    'task': 'classification',
    'primary_metric': 'AUC_weighted',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    training_data=train_data,
    validation_data=val_data,
    label_column_name='y',
    compute_target=compute_target,
    **automl_settings)



In [11]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on comp-capstone with default configuration
Running on remote compute: comp-capstone


Experiment,Id,Type,Status,Details Page,Docs Page
Capstone-AutoML,AutoML_683a1772-a4d9-48e6-9620-4b70c81abe0a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: ht

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [22]:
best_run_automl = automl_run.get_best_child()



In [23]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/65_job_prep-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/105_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [24]:
best_run_automl.download_file(best_run_automl.get_file_names()[-3])


In [25]:
# Save the downloaded file to local storage
local_path='./outputs'
os.makedirs(local_path, exist_ok=True)

best_automl_model = joblib.load(open(best_run_automl.get_file_names()[-3].split('/')[1], 'rb'))
joblib.dump(best_automl_model, 'outputs/best_automl_model.joblib')

['outputs/best_automl_model.joblib']

In [26]:
best_run_automl.get_metrics()

{'recall_score_micro': 0.7890547263681592,
 'balanced_accuracy': 0.789019801980198,
 'f1_score_weighted': 0.7890409405151873,
 'AUC_micro': 0.8673885409877093,
 'weighted_accuracy': 0.7890896490272759,
 'average_precision_score_weighted': 0.8638382010353565,
 'average_precision_score_micro': 0.8641698459930782,
 'accuracy': 0.7890547263681592,
 'norm_macro_recall': 0.5780396039603961,
 'log_loss': 0.49779187078562687,
 'AUC_weighted': 0.867168316831683,
 'matthews_correlation': 0.578115164543608,
 'precision_score_macro': 0.7890953675020005,
 'f1_score_macro': 0.7890294523043774,
 'precision_score_micro': 0.7890547263681592,
 'average_precision_score_macro': 0.8638582900769595,
 'precision_score_weighted': 0.7890834142273412,
 'recall_score_weighted': 0.7890547263681592,
 'AUC_macro': 0.867168316831683,
 'f1_score_micro': 0.7890547263681592,
 'recall_score_macro': 0.789019801980198,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_683a1772-a4d9-48e6-9620-4b70c81abe0a_29

In [27]:
X_test

array([[ 1,  0,  0, ...,  0,  0,  0],
       [12,  0,  1, ...,  0,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [30]:
test_dataset = pd.DataFrame(X_test)

In [33]:
test_dataset.columns = "X" + test_dataset.columns.astype(str)

In [34]:
pred_test = best_automl_model.predict(test_dataset)

In [35]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_test, pred_test)
accuracy = accuracy_score(y_test, pred_test)

print('AUC Test:', auc_score)
print('Accuracy Test:', accuracy)

AUC Test: 0.7645762644831458
Accuracy Test: 0.7644444444444445


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/65_job_prep-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_f8c43420721b6df8cab68fd148a2a5f7aaf79cb4016c74a657042736e9a99e6a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/105_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [37]:
print(best_run_automl.properties['model_name'])
model_name = best_run_automl.properties['model_name']

registered_model = automl_run.register_model(model_name=model_name,
description="Stackoverflow Quality Prediction", tags=None)

AutoML683a1772a29


In [38]:
local_path='./inference'
os.makedirs(local_path, exist_ok=True)

best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [39]:
inference_config = InferenceConfig(entry_script="inference/score.py")

In [40]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2, description="Stackoverflow comment quality prediction")

In [41]:
service = Model.deploy(ws, "automl-stackoverflow", [registered_model], inference_config, aciconfig)
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-15 02:31:47+00:00 Creating Container Registry if not exists..
2021-05-15 02:31:57+00:00 Use the existing image.
2021-05-15 02:31:57+00:00 Generating deployment configuration..
2021-05-15 02:31:58+00:00 Submitting deployment to compute.
2021-05-15 02:32:03+00:00 Checking the status of deployment automl-stackoverflow..
2021-05-15 02:35:38+00:00 Checking the status of inference endpoint automl-stackoverflow.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [47]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-144838', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-144838'), name=automl-stackoverflow, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://99bfe646-821c-448d-92a0-80d5a3ebcd1a.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

In [86]:
import random
import json

sample_dict = {}
for i in range(287):
    
    sample_dict["X"+str(i)] = random.choice([0,1])
sample_json = json.dumps({"data":[sample_dict,]})

In [67]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [93]:
import requests
rest_endpoint = "http://99bfe646-821c-448d-92a0-80d5a3ebcd1a.southcentralus.azurecontainer.io/score"
headers = {"Content-Type": "application/json"}

response = requests.post(rest_endpoint, sample_json, headers=headers)

In [94]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

print(response.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [95]:
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "automl-stackoverflow"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

# Solution code: Enable application insights
service.update(enable_app_insights=True)

for line in logs.split('\n'):
    print(line)


2021-05-15T02:35:27,922953309+00:00 - gunicorn/run 
2021-05-15T02:35:27,924364314+00:00 - iot-server/run 
2021-05-15T02:35:27,925720019+00:00 - rsyslog/run 
2021-05-15T02:35:27,937128961+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_98cae94c606e3ceb655a787040a8a93c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
service